In [55]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import  r2_score

import lightgbm as lgb
import xgboost as xgb
from subprocess import check_output
print(check_output(['ls','input']).decode('utf8'))

sample_submission.csv
test.csv
train.csv



In [56]:
train = pd.read_csv('input/train.csv', parse_dates=['start_date','creation_date','sell_date'])
test = pd.read_csv('input/test.csv', parse_dates=['start_date','creation_date','sell_date'])

print('Train data size: {} Test data size: {}'.format(train.shape, test.shape))

Train data size: (9366, 18) Test data size: (4801, 17)


In [57]:
train['desk_id'].fillna('DSK00000099', inplace=True)
test['desk_id'].fillna('DSK00000099', inplace=True)

train['desk_id']=train.desk_id.apply(lambda x : x[-4:]).astype(np.int16)
test['desk_id']=test.desk_id.apply(lambda x : x[-4:]).astype(np.int16)

In [58]:
le = LabelEncoder()
le.fit(train['office_id'])
train['office_id'] = le.transform(train['office_id'])
test['office_id'] = le.transform(test['office_id'])

In [59]:
le.fit(train['pf_category'])
train['pf_category'] = le.transform(train['pf_category'])
test['pf_category'] = le.transform(test['pf_category'])

In [60]:
train['start_creation'] = (train.start_date - train.creation_date).dt.days
test['start_creation'] = (test.start_date - test.creation_date).dt.days

In [61]:
train['start_sell'] = (train.start_date - train.sell_date).dt.days
test['start_sell'] = (test.start_date - test.sell_date).dt.days

In [62]:
#train['creation_sell'] = np.abs(train.creation_date - train.sell_date).dt.days
#test['creation_sell'] = np.abs(test.creation_date - test.sell_date).dt.days

In [63]:
train['sold'].fillna(68552000.0, inplace= True)
train['bought'].fillna(68427270.0, inplace= True)

In [64]:
train['dff_sold_bought'] = (train.sold - train.bought)
test['dff_sold_bought'] = (test.sold - test.bought)

In [65]:
#train['gain_loss'] = train.dff_sold_bought * train.euribor_rate 
#test['gain_loss'] = test.dff_sold_bought * test.euribor_rate 

In [66]:
train.drop(['start_date','creation_date', 'sell_date'], inplace=True, axis=1)
test.drop(['start_date','creation_date', 'sell_date'], inplace=True, axis=1)

In [67]:
le.fit(train['country_code'])
train['country_code'] = le.transform(train['country_code'])
test['country_code'] = le.transform(test['country_code'])

In [68]:
le.fit(train['currency'])
train['currency'] = le.transform(train['currency'])
test['currency'] = le.transform(test['currency'])

In [69]:
train['libor_rate'].fillna(0.0, inplace= True)
test['libor_rate'].fillna(0.0, inplace=True)

In [70]:
train['indicator_code'].fillna(False, inplace= True)
test['indicator_code'].fillna(False, inplace= True)

le.fit(train['indicator_code'])
train['indicator_code'] = le.transform(train['indicator_code'])
test['indicator_code'] = le.transform(test['indicator_code'])

In [71]:
le.fit(train['type'])
train['type'] = le.transform(train['type'])
test['type'] = le.transform(test['type'])

In [72]:
train['hedge_value'].fillna(False, inplace=True)
test['hedge_value'].fillna(False, inplace=True)

train['hedge_value'] = train['hedge_value'].map({False:0, True:1})
test['hedge_value'] = test['hedge_value'].map({False:0, True:1})

In [73]:
train['status'].fillna(False, inplace= True)
test['status'].fillna(False, inplace= True)

train['status'] = train['status'].map({False:0, True:1})
train['status'] = train['status'].map({False:0, True:1})

In [74]:
retn = train['return']
test_portfolio_id = test.portfolio_id.values

train.drop(['portfolio_id','bought', 'type', 'return'], inplace=True, axis=1)
test.drop(['portfolio_id','bought', 'type'], inplace=True, axis=1)

In [87]:
params = {'learning_rate': 0.02, 
          'num_leaves':78,
          'min_data_in_leaf': 260,
          'max_depth': 12, 
          'colsample_bytree': 0.522,
          'boosting': 'gbdt', 
          'application': 'regression', 
          'metric': 'mae',  
          'verbosity': -1,
          'seed': 32}

In [88]:
lgb_train = lgb.Dataset(train, label=np.log1p(retn))

In [93]:
lgb_cv = lgb.cv(params, lgb_train, num_boost_round=10000, nfold= 5, stratified=False,
                 verbose_eval=20, early_stopping_rounds=40)

[20]	cv_agg's l1: 0.00975097 + 8.86842e-05
[40]	cv_agg's l1: 0.00691001 + 7.82472e-05
[60]	cv_agg's l1: 0.00513465 + 7.82855e-05
[80]	cv_agg's l1: 0.00395018 + 8.20111e-05
[100]	cv_agg's l1: 0.00311514 + 8.94672e-05
[120]	cv_agg's l1: 0.00258169 + 8.88756e-05
[140]	cv_agg's l1: 0.00220314 + 9.42904e-05
[160]	cv_agg's l1: 0.00197024 + 9.16449e-05
[180]	cv_agg's l1: 0.00182604 + 8.88876e-05
[200]	cv_agg's l1: 0.00173286 + 8.85486e-05
[220]	cv_agg's l1: 0.00168375 + 8.88603e-05
[240]	cv_agg's l1: 0.00163989 + 8.75692e-05
[260]	cv_agg's l1: 0.00160298 + 8.79129e-05
[280]	cv_agg's l1: 0.00157484 + 8.89602e-05
[300]	cv_agg's l1: 0.0015509 + 8.92622e-05
[320]	cv_agg's l1: 0.00153007 + 8.99209e-05
[340]	cv_agg's l1: 0.00151212 + 8.9069e-05
[360]	cv_agg's l1: 0.00149036 + 8.91713e-05
[380]	cv_agg's l1: 0.00147495 + 9.16905e-05
[400]	cv_agg's l1: 0.00145979 + 9.09037e-05
[420]	cv_agg's l1: 0.00144374 + 9.3571e-05
[440]	cv_agg's l1: 0.00143009 + 9.24275e-05
[460]	cv_agg's l1: 0.001422 + 9.2745e-0

In [94]:
nround = lgb_cv['l1-mean'].index(np.min(lgb_cv['l1-mean']))
print(nround)

1796


In [95]:
model = lgb.train(params, lgb_train, num_boost_round=nround)

In [96]:
r2_score(retn, np.expm1(model.predict(train)))

0.85320424373033599

In [97]:
pred = model.predict(test)

In [98]:
sub = pd.DataFrame({'portfolio_id':test_portfolio_id, 'return':np.expm1(pred)})
sub.head(5)

,portfolio_id,return
0,PF00001001,0.028845
1,PF00001004,0.033722
2,PF00001009,0.024596
3,PF00001013,0.026988
4,PF00001014,0.023472


In [99]:
sub.to_csv('lgb_v2.csv', index=False)

In [100]:
def xgb_r2(pred, drain):
    y = list(drain.get_label())
    score = r2_score(y, np.expm1(pred))
    return 'r2_score', score
    

In [101]:
xgb_params = {
    'seed': 4,
    'colsample_bytree': 0.73,
    'silent': 1,
    'subsample': 0.63,
    'learning_rate': 0.01,
    'objective':'reg:linear',    
    'max_depth': 7,
    'num_parallel_tree':8 ,
    'min_child_weight': 2,
    #'gamma': 0.046,
    #'reg_alpha': 1.39,    
}

In [102]:
dtrain = xgb.DMatrix(data=train, label=np.log1p(retn))
dtest = xgb.DMatrix(data=test)

In [103]:
res = xgb.cv(xgb_params, dtrain, num_boost_round=2000, nfold=5,  stratified=False, feval = xgb_r2,
             early_stopping_rounds=25, verbose_eval=20, show_stdv=True,  maximize=True)


[0]	train-r2_score:-1057.66+45.5899	test-r2_score:-1079.6+145.014
[20]	train-r2_score:-644.027+27.7589	test-r2_score:-657.404+88.4058
[40]	train-r2_score:-399.219+17.2041	test-r2_score:-407.525+54.8704
[60]	train-r2_score:-250.984+10.815	test-r2_score:-256.217+34.5452
[80]	train-r2_score:-159.544+6.87123	test-r2_score:-162.88+21.9985
[100]	train-r2_score:-102.279+4.40321	test-r2_score:-104.426+14.1313
[120]	train-r2_score:-65.973+2.83992	test-r2_score:-67.3654+9.13772
[140]	train-r2_score:-42.7192+1.84063	test-r2_score:-43.6295+5.93538
[160]	train-r2_score:-27.7025+1.19594	test-r2_score:-28.303+3.86464
[180]	train-r2_score:-17.9395+0.777276	test-r2_score:-18.3391+2.51526
[200]	train-r2_score:-11.5541+0.504251	test-r2_score:-11.8239+1.63034
[220]	train-r2_score:-7.35803+0.325444	test-r2_score:-7.54363+1.04751
[240]	train-r2_score:-4.58883+0.208328	test-r2_score:-4.72149+0.662229
[260]	train-r2_score:-2.75487+0.131274	test-r2_score:-2.85441+0.406629
[280]	train-r2_score:-1.53578+0.081009

In [104]:
best_nrounds = res.shape[0] - 1
cv_mean = res.iloc[-1, 0]
cv_std = res.iloc[-1, 1]

print('xgb-CV: {0}+{1}'.format(cv_mean, cv_std))

xgb-CV: 0.8519137999999999+0.13659603561070136


In [105]:
gbdt = xgb.train(xgb_params, dtrain, best_nrounds)

In [106]:
pred_1 = gbdt.predict(dtest)

In [107]:
sub1 = pd.DataFrame({'portfolio_id':test_portfolio_id, 'return':np.expm1(pred_1)})
sub1.head(5)

,portfolio_id,return
0,PF00001001,0.026341
1,PF00001004,0.032002
2,PF00001009,0.025321
3,PF00001013,0.025911
4,PF00001014,0.024794


In [108]:
sub1.to_csv('xgb_v1.csv', index=False)

In [112]:
pred2 = np.expm1(0.7*pred + 0.3*pred_1)

In [113]:
sub2 = pd.DataFrame({'portfolio_id':test_portfolio_id, 'return':pred2})
sub2.head(5)

,portfolio_id,return
0,PF00001001,0.028093
1,PF00001004,0.033205
2,PF00001009,0.024813
3,PF00001013,0.026665
4,PF00001014,0.023868


In [114]:
sub2.to_csv('lgb_xgb_v2.csv', index=False)